### This is the basic idea and code snippets of our MWPM decoder.
first need to set some parameter for algorithm

In [3]:
p_x = 0.05
p_y = 0.05
p_z = 0.05
local_search_m = 25 # the maximun degree when apply local dijkstra to build syndrome graph

#### 1. Read the testpoint

In [32]:
import numpy as np

# error_x, error_z, syndrome_x, syndrome_z,sizeX,sizeY = read_test_file('test_file.npz')
def read_test_file(filename):
    file_path = "./data/input/" + filename
    data = np.load(file_path, allow_pickle=True)
    file_content = data['arr_0']
    return file_content

# Here we only test
testset_list = []
test_batches = 20
testset_num = 200

for i in range(test_batches):
    testset_list.append(read_test_file("test"+str(i)+".npz"))

def pack_one_result(consume_time,correct_x,correct_z):
    file_content = {
        "time": consume_time,
        "correct_x":correct_x,
        "correct_z":correct_z,
    }
    return file_content

def write_result_file(content_list,filename):
    np.savez(".output/"+filename, content_list)

#### 2. Build Matching graph
apply weight to every qubit like this:
$
w_i = \log\left(\frac{1 - p_i}{p_i}\right)
$

where $p_i$ is the physical error probability of ith qubit. So here generate all weight first.


In [9]:
from math import log2
# generate weight matrix for every qubit, this is not precise.
def get_weight(p_x):
    return log2((1-p_x)/p_x)
def build_matching_graph(sizeX,sizeY,p):
    #it is a adjacency matrix, the weight of edge is the log2 of the probability of the edge
    weight_mat = np.full((sizeX*sizeY,sizeX*sizeY), 99999)
    
    #only the adjacent vertex(4 directions) has normal weight, for toric code there is no edge
    for i in range(sizeX*sizeY):
        if i%sizeX != 0: # not the left edge
            weight_mat[i][i-1] = get_weight(p)
        else:
            weight_mat[i][i+sizeX-1] = get_weight(p)
        if i%sizeX != sizeX-1: # not the right edge
            weight_mat[i][i+1] = get_weight(p)
        else:
            weight_mat[i][i-sizeX+1] = get_weight(p)
        if i//sizeX != 0: # not the top row
            weight_mat[i][i-sizeX] = get_weight(p)
        else:
            weight_mat[i][i+sizeX*(sizeY-1)] = get_weight(p)
        if i//sizeX != sizeY-1: # not the bottom row
            weight_mat[i][i+sizeX] = get_weight(p)
        else:
            weight_mat[i][i-sizeX*(sizeY-1)] = get_weight(p)
    for i in range (sizeX*sizeY):
        weight_mat[i][i] = 0
    return weight_mat
# print(build_matching_graph(3,3,0.05))

#### 3. Build Syndrome graph

using dijkstra / local dijkstra to build a syndrome graph

There is an method to make it faster, local dijkstra.

because we will use dijkstra algorithm also in blossom algorithm, so build adjacent matrix for syndrome graph

In [87]:
import numpy as np

def list_insert(list,pair):
    insert_index=0
    len_list = len(list)
    while(insert_index<len_list and list[insert_index][0] < pair[0]):
        insert_index+=1
    list.insert(insert_index,pair)

def list_remove_index(list,index):
    for i in range(len(list)):
        if list[i][1] == index:
            list.pop(i)
            break

#matching_graph is a adjacent matrix
#syndrome is a 1D vector, 1 means defect, 0 means no defect
#use local_search_m as minimun degree of each defect node.
#the start node is a index, from 0 to sizeX*sizeY-1
def local_dijkstra(matching_graph,syndrome,local_search_m,start,end=None): 
    # Create a priority queue
    dist_from_start = np.full(len(matching_graph), 99999) # the distance from start to each node
    dist_from_start[start] = 0
    # need to record the path
    parents = np.full(len(matching_graph),-1)
    pq = []
    pq.append((0,start))

    found_defects = [] # the defects found by local dijkstra, decide the min degree of the start node
    while pq and len(found_defects) < local_search_m:
        #find minimun distance node
        current = pq.pop(0)[1]

        if syndrome[current] == 1: # find a defect node
            found_defects.append(current)
        
        for i in range(len(matching_graph)):
            if matching_graph[current][i] == 0 or matching_graph[current][i] == 99999: # not the neighbor
                continue
            if dist_from_start[current] + matching_graph[current][i] < dist_from_start[i]: # update the distance
                dist_from_start[i] = dist_from_start[current] + matching_graph[current][i]
                parents[i] = current
                if(dist_from_start[i] == 99999): #never visit
                    list_insert(pq,(dist_from_start[i],i))
                else: # visit, need to decrease value, here should be decrease key.
                    list_remove_index(pq,i)
                    list_insert(pq,(dist_from_start[i],i))
    # get the path from start to end
    if(end!=None):
        path = [end]
        curnode = end
        while(parents[curnode]!=-1):
            curnode = parents[curnode]
            path.append(curnode)
        return path
    return found_defects,dist_from_start

def build_syndrome_graph(matching_graph,syndrome,local_search_m):
    # extract every defect node, and form a graph
    syndrome_nodes = np.where(syndrome == 1)[0]
    # generate the syndrome graph
    len_syndrome_nodes = len(syndrome_nodes)
    adj_matrix = np.zeros((len_syndrome_nodes,len_syndrome_nodes))
    for i in range(len_syndrome_nodes): # start from every defect node
        found_defects,dist_from_start = local_dijkstra(matching_graph,syndrome,local_search_m,syndrome_nodes[i])
        for j in range(len(found_defects)): # update the weight of the edge
            adj_matrix[i][np.argmax(syndrome_nodes==found_defects[j])] = dist_from_start[found_defects[j]]
    return adj_matrix,syndrome_nodes

# adj_matrix,syndrome_nodes = build_syndrome_graph(build_matching_graph(3,3,0.05),np.array([[1,0,0],[0,1,1],[0,0,1]]),local_search_m)
# print(adj_matrix)


#### 4. Blossom V
use blossom V algorithm to find perfect matching

In [88]:
from blossom import Blossom
# a small test of blossom algorithm
gh=[[0 for _ in range(5)] for _ in range(5)]
gh[1][2]=gh[2][1]=4
gh[1][3]=gh[3][1]=2
gh[1][4]=gh[4][1]=6
gh[2][3]=gh[3][2]=1
gh[2][4]=gh[4][2]=3
gh[3][4]=gh[4][3]=5
print(gh)

[[0, 0, 0, 0, 0], [0, 0, 4, 2, 6], [0, 4, 0, 1, 3], [0, 2, 1, 0, 5], [0, 6, 3, 5, 0]]


In [89]:
mp=Blossom(gh,4)
print(mp)

[(1, 3), (2, 4)]


#### 5. output correct graph
Need to output every edge(sizeX\*sizeY\*2) on the matching graph, tell whether we detect error there.

In [92]:
import time

for testset_index in range(test_batches):
    result_list = []
    for test_index in range(testset_num):
        # read in
        test = testset_list[testset_index][test_index]
        sizeX = test["sizeX"]
        sizeY = test["sizeY"]
        # turn syndrome graph into vector, so it will be easier to use local dijkstra
        syndrome_z =  test["syndrome_z"].flatten()
        time_start = time.time()
        matching_graph = build_matching_graph(sizeX, sizeY, p_x)
        adj_matrix, syndrome_nodes = build_syndrome_graph(matching_graph ,syndrome_z, local_search_m)
        #create padding for blossom
        len_syndrome_nodes = len(syndrome_nodes)
        padded_matrix = np.zeros((len_syndrome_nodes + 1, len_syndrome_nodes + 1), dtype=adj_matrix.dtype)
        padded_matrix[1:, 1:] = adj_matrix

        print(padded_matrix)

        matching_result = Blossom(padded_matrix, len_syndrome_nodes)
        time_spend = time.time() - time_start
        
        #extract result
        correct_z = np.zeros((sizeY*2,sizeX),dtype=int)
        # for each pair in match result, generate the correct z
        for u,v in matching_result:
            if u == 0 or v == 0:
                continue
            u = syndrome_nodes[u-1] # get the real index in lattice
            v = syndrome_nodes[v-1]
            # find the edge between u and v
            path = local_dijkstra(matching_graph,syndrome_z,local_search_m,u,v)
            #add path to the correct_z
            for index in path:
                correct_z[index//sizeX][index%sizeX] = 1
        result_list.append(pack_one_result(time_spend,{},correct_z))
        
    #write result
    


[[0. 0. 0. 0. 0.]
 [0. 0. 4. 4. 8.]
 [0. 4. 0. 8. 4.]
 [0. 4. 8. 0. 4.]
 [0. 8. 4. 4. 0.]]
[[0.]]
[[0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 8. 8. 4.]
 [0. 8. 0. 4. 8.]
 [0. 8. 4. 0. 4.]
 [0. 4. 8. 4. 0.]]
[[0.]]
[[0.]]
[[0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 4. 8. 8. 4. 8.]
 [0. 4. 0. 8. 4. 8. 8.]
 [0. 8. 8. 0. 4. 8. 4.]
 [0. 8. 4. 4. 0. 8. 8.]
 [0. 4. 8. 8. 8. 0. 4.]
 [0. 8. 8. 4. 8. 4. 0.]]
[[0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 4. 8. 8.]
 [0. 4. 0. 8. 4.]
 [0. 8. 8. 0. 4.]
 [0. 8. 4. 4. 0.]]
[[0.]]
[[0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[0. 0. 0.]
 [0. 0. 4.]
 [0. 4. 0.]]
[[

IndexError: list assignment index out of range